# Setup

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
from scipy.stats import linregress
from scipy.stats import sem
import numpy as np

# Extracting and Transforming (cleansing) the COVID-19_Hospital_Data_Coverage_Detail dataset

In [62]:
file_path = "Resources/COVID-19_Hospital_Data_Coverage_Detail.csv"

In [7]:
file_df = pd.read_csv(file_path )


file_df.head()


,state,ccn,facility_name,street_address,city,zip_code,fips_county,provider_subtype,reporting_source,dates_with_missing_data,...,on_hand_supply_therapeutic_b_bamlanivimab_courses,previous_week_therapeutic_b_bamlanivimab_courses_used,previous_week_personnel_covid_vaccinated_doses_administered,total_personnel_covid_vaccinated_doses_none,total_personnel_covid_vaccinated_doses_one,total_personnel_covid_vaccinated_doses_all,total_personnel,previous_week_patients_covid_vaccinated_doses_one,previous_week_patients_covid_vaccinated_doses_all,Georeference
0,AR,040007,CHI-ST VINCENT INFIRMARY,TWO ST VINCENT CIRCLE,LITTLE ROCK,72205.0,119,Short Term,tt,[],...,True,True,True,True,True,True,True,True,True,NaN
1,AR,040161,ARKANSAS HEART HOSPITAL-ENCORE,1901 ENCORE WAY,BRYANT,72022.0,125,Short Term,tt,"['2021-12-01', '2021-12-02']",...,False,False,False,False,False,False,False,False,False,NaN
2,AR,042009,REGENCY HOSPITAL OF NORTHWEST ARKANSAS,609 WEST MAPLE AVENUE,SPRINGDALE,72764.0,143,Long Term,tt,[],...,True,True,False,False,False,False,False,False,False,POINT (-94.136464 36.181394)
3,CT,070016,SAINT MARY'S HOSPITAL,56 FRANKLIN STEET,WATERBURY,6706.0,9,Short Term,tt,[],...,True,True,False,False,False,False,False,False,False,NaN
4,IA,161342,MONROE COUNTY HOSPITAL,6580 165TH STREET,ALBIA,52531.0,135,Critical Access Hospitals,tt,[],...,False,False,False,False,False,False,False,False,False,POINT (-92.796674 41.044816)


In [8]:
file_df.count()

state                                                5955
ccn                                                  5955
facility_name                                        5955
street_address                                       5955
city                                                 5955
                                                     ... 
total_personnel_covid_vaccinated_doses_all           5955
total_personnel                                      5955
previous_week_patients_covid_vaccinated_doses_one    5955
previous_week_patients_covid_vaccinated_doses_all    5955
Georeference                                         5282
Length: 97, dtype: int64

In [12]:
file_df.columns

Index(['state', 'ccn', 'facility_name', 'street_address', 'city', 'zip_code',
       'fips_county', 'provider_subtype', 'reporting_source',
       'dates_with_missing_data', 'certified_bed_count', 'component_hospitals',
       'week_ending', 'days_at_100%', 'percentage_of_required_fields_reported',
       'hospital_required_to_report_wed_only', 'total_beds',
       'all_adult_hospital_beds', 'inpatient_beds',
       'all_adult_hospital_inpatient_beds', 'inpatient_beds_used',
       'all_adult_hospital_inpatient_bed_occupied', 'total_icu_beds',
       'total_staffed_adult_icu_beds', 'icu_beds_used',
       'staffed_adult_icu_bed_occupancy', 'total_ventilators',
       'ventilators_used',
       'total_adult_patients_hospitalized_confirmed_and_suspected_covid',
       'total_adult_patients_hospitalized_confirmed_covid',
       'total_pediatric_patients_hospitalized_confirmed_and_suspected_covid',
       'total_pediatric_patients_hospitalized_confirmed_covid',
       'inpatient_ventilator

In [69]:
file_clean_df = file_df[["state", "facility_name", "street_address", "city", "zip_code", "week_ending", "certified_bed_count", "total_beds", "all_adult_hospital_beds", "inpatient_beds", "all_adult_hospital_inpatient_beds", "inpatient_beds_used", "all_adult_hospital_inpatient_bed_occupied", "total_icu_beds", "icu_beds_used", "total_adult_patients_hospitalized_confirmed_covid", "total_pediatric_patients_hospitalized_confirmed_and_suspected_covid","total_pediatric_patients_hospitalized_confirmed_covid", "staffed_icu_adult_patients_confirmed_and_suspected_covid", "staffed_icu_adult_patients_confirmed_covid", "deaths_covid"]]

file_clean_df.head()

,state,facility_name,street_address,city,zip_code,week_ending,certified_bed_count,total_beds,all_adult_hospital_beds,inpatient_beds,...,inpatient_beds_used,all_adult_hospital_inpatient_bed_occupied,total_icu_beds,icu_beds_used,total_adult_patients_hospitalized_confirmed_covid,total_pediatric_patients_hospitalized_confirmed_and_suspected_covid,total_pediatric_patients_hospitalized_confirmed_covid,staffed_icu_adult_patients_confirmed_and_suspected_covid,staffed_icu_adult_patients_confirmed_covid,deaths_covid
0,AR,CHI-ST VINCENT INFIRMARY,TWO ST VINCENT CIRCLE,LITTLE ROCK,72205.0,12/02/2021 12:00:00 AM,615,7.0,7.0,7.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
1,AR,ARKANSAS HEART HOSPITAL-ENCORE,1901 ENCORE WAY,BRYANT,72022.0,12/02/2021 12:00:00 AM,25,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
2,AR,REGENCY HOSPITAL OF NORTHWEST ARKANSAS,609 WEST MAPLE AVENUE,SPRINGDALE,72764.0,12/02/2021 12:00:00 AM,25,7.0,7.0,7.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
3,CT,SAINT MARY'S HOSPITAL,56 FRANKLIN STEET,WATERBURY,6706.0,12/02/2021 12:00:00 AM,347,7.0,7.0,7.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
4,IA,MONROE COUNTY HOSPITAL,6580 165TH STREET,ALBIA,52531.0,12/02/2021 12:00:00 AM,25,7.0,7.0,7.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0


# Cleansed dataset to be loaded

In [72]:
file_clean_df.to_csv('Resources/COVID-19_Hospital_Data_Coverage_Detail_cleansed.csv', index=False)

# Appendix

In [71]:
file_clean_df1 = file_clean_df.loc[(file_clean_df["state"] == 'NY') | (file_clean_df["state"] == 'CA')]

file_clean_df1.head()

,state,facility_name,street_address,city,zip_code,week_ending,certified_bed_count,total_beds,all_adult_hospital_beds,inpatient_beds,...,inpatient_beds_used,all_adult_hospital_inpatient_bed_occupied,total_icu_beds,icu_beds_used,total_adult_patients_hospitalized_confirmed_covid,total_pediatric_patients_hospitalized_confirmed_and_suspected_covid,total_pediatric_patients_hospitalized_confirmed_covid,staffed_icu_adult_patients_confirmed_and_suspected_covid,staffed_icu_adult_patients_confirmed_covid,deaths_covid
8,NY,UNITY SPECIALTY HOSPITAL,89 GENESEE STREET,ROCHESTER,14611.0,12/02/2021 12:00:00 AM,40,6.0,6.0,6.0,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0
59,CA,UCSF BENIOFF CHILDREN'S HOSPITAL OAKLAND,747 52ND STREET,OAKLAND,94609.0,12/02/2021 12:00:00 AM,190,6.0,6.0,6.0,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0
61,CA,SHARP MESA VISTA HOSPITAL,7850 VISTA HILL AVENUE,SAN DIEGO,92123.0,12/02/2021 12:00:00 AM,149,Yes,Yes,Yes,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes
67,CA,KAISER FOUNDATION HOSPITAL FONTANA,9961 SIERRA AVE,FONTANA,92335.0,12/02/2021 12:00:00 AM,438,7.0,7.0,7.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
68,CA,GATEWAYS HOSP MENTAL HEALTH CTR,1891 EFFIE ST,LOS ANGELES,90026.0,12/02/2021 12:00:00 AM,55,No,No,No,...,No,No,No,No,No,No,No,No,No,No
